In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from os.path import join
import csv
from tqdm.auto import tqdm
import re
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler , PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.base import BaseEstimator, RegressorMixin
import statsmodels.api as sm
from xgboost import XGBRegressor
from Feature_engineering.FE import *
from Feature_engineering.utils import *
from Feature_engineering.ProjectPaths import *

In [3]:
golden_df=pd.read_csv(join(output_folder_paths,"golden.csv"))
eugene_df=pd.read_csv(join(output_folder_paths,"eugene.csv"))
cocoa_df=pd.read_csv(join(output_folder_paths,"cocoa.csv"))

In [4]:
main_df=pd.concat([golden_df,eugene_df,cocoa_df],axis=0)

In [5]:
main_df["filename"]=main_df["filename"].apply(lambda x : x.split("_")[-1])

In [6]:
main_df,columns,lickage_features=feature_eng(main_df)

removing these columns :  []


In [7]:
len(main_df)

769595

In [8]:
sites=main_df["filename"].unique()

In [9]:
X=main_df.drop(columns=["Imp (A)","Pmp (W)","Vmp (V)"]+lickage_features)
Y=main_df[["Imp (A)","Pmp (W)","Vmp (V)","filename"]]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42,stratify=X["filename"])

In [10]:
x_train_sites=split_by_site(x_train,sites)
x_test_sites=split_by_site(x_test,sites)
y_train_sites=split_by_site(y_train,sites)
y_test_sites=split_by_site(y_test,sites)

In [11]:
xg_boost_params = {
    "n_estimators": 500,
    "max_depth": 3,
    "min_samples_split": 7,
    "learning_rate": 0.01,
    "loss": "squared_error",
    "verbose":0
}
l1_xg_boost_params={
    "objective":'reg:squarederror',
    "n_estimators": 500,
    "max_depth": 3,
    "learning_rate": 0.01,
    "reg_alpha":2.0,
}
mlp_params={
    "hidden_layer_sizes":(128, 64,16),
    "activation":'relu',
    "solver":'adam',
    "learning_rate_init":0.01,
    "max_iter":500,
    "random_state":42,
    "verbose":0
}
forest_params={
    "max_depth":10, 
    "random_state":0,
    "min_samples_split":10,
    "min_samples_leaf":10, 
    "max_features":'sqrt'
}

# training_labels=["Imp (A)","Pmp (W)","Vmp (V)"]

In [12]:
ip_features=["GHI","DNI","POA_CMP22","Mod_Temp"]
ip_labels=["Imp (A)","Pmp (W)"]
v_features=["Cab_Temp","Dry_Temp","Mod_Temp","Pressure","RH","QA_residual","GHI","DNI"]
preprocessor = ColumnTransformer([
    ("norm", StandardScaler(),ip_features)
])
v_preprocessor = ColumnTransformer([
    ("norm", StandardScaler(),v_features+["i","p"]) #list(range(len(v_features)+2))
])
def multi_train(reg_model,v_reg_model,stat_rep=False):

    vmp_test_r2_ls=[]
    vmp_train_r2_ls=[]
    adj_vmp_test_r2_ls=[]
    adj_vmp_train_r2_ls=[]
    for solar_model in x_train_sites:

        x_tr=x_train_sites[solar_model]
        x_te=x_test_sites[solar_model]
        y_tr=y_train_sites[solar_model]
        y_te=y_test_sites[solar_model]

        reg_model.fit(x_tr[ip_features], y_tr[ip_labels])
        ip_preds=reg_model.predict(x_tr[ip_features])
        ip_preds=pd.DataFrame(np.concatenate((ip_preds,x_tr[v_features].to_numpy()),axis=1),
                              columns=v_features+["i","p"])
        v_reg_model.fit(ip_preds,y_tr["Vmp (V)"])
        
        print(f"for site {solar_model}")
        
        test_ip_preds=reg_model.predict(x_te[ip_features])
        test_ip_preds=pd.DataFrame(np.concatenate((test_ip_preds,x_te[v_features].to_numpy()),axis=1),
                              columns=v_features+["i","p"])
        
        vmp_test_r2,vmp_train_r2,adj_vmp_train_r2,adj_vmp_test_r2=evaluate(v_reg_model,ip_preds,y_tr[["Vmp (V)"]],
                                                                           test_ip_preds,y_te[["Vmp (V)"]])

        vmp_test_r2_ls.append(vmp_test_r2)
        vmp_train_r2_ls.append(vmp_train_r2)
        adj_vmp_train_r2_ls.append(adj_vmp_train_r2)
        adj_vmp_test_r2_ls.append(adj_vmp_test_r2)
        if(stat_rep):
            v_reg_model.named_steps["model"].get_f_test()
        print("-"*100)
        
        if(solar_model == "mSi460BB" and stat_rep):
            names=get_f_names(v_reg_model)
            print(v_reg_model.named_steps["model"].get_summary(names))
    print(f"mean test r2 : {np.mean(vmp_test_r2_ls)} | mean train r2 :{np.mean(vmp_train_r2_ls)}")  
    print(f"mean test adj-r2 : {np.mean(adj_vmp_test_r2_ls)} | mean train adj-r2 :{np.mean(adj_vmp_train_r2_ls)}") 
#aSiTandem90-31 # aSiMicro03038

In [15]:
class StatsModelsOLS(BaseEstimator, RegressorMixin):
    def __init__(self):
        pass
    def fit(self, X, y):
        X_with_const = sm.add_constant(X, has_constant='add')
        self.results_= sm.OLS(y, X_with_const).fit()
        params =self.results_.params
        # print(params.shape)
        self.intercept_=params[0]
        self.coef_ = np.asarray(params[1:])
        return self

    def predict(self, X):
        return self.intercept_ + np.dot(X, self.coef_)
    def get_f_test(self):
        f_statistic = self.results_.fvalue
        f_pvalue= self.results_.f_pvalue
        print(f"f statistics : {f_statistic} | f_pvalue : {f_pvalue}")
    def get_summary(self,names):
        return self.results_.summary(xname=["int"]+names.tolist())
def get_f_names(pipe):
    feat_names_pre=pipe.named_steps["preproc"].get_feature_names_out()
    if("poly" in pipe.named_steps):
        feat_names_pre = pipe.named_steps["poly"].get_feature_names_out(feat_names_pre)
    return feat_names_pre

In [29]:
reg_model=Pipeline([
    ("preproc", preprocessor),
    ("poly",PolynomialFeatures(degree=2)),
    ("model", LinearRegression())
])
v_reg_model=Pipeline([
    ("preproc", v_preprocessor),
    ("poly",PolynomialFeatures(degree=2)),
    ("model", StatsModelsOLS()) #XGBRegressor(**l2_xg_boost_params)
])
multi_train(reg_model,v_reg_model,stat_rep=True)

for site mSi0247
R2 For Vmp (V) : 
Test R2 : 0.9569205497675896 | Train R2 : 0.9529339171595328
Test ADJ-R2 : 0.9566067882946659 | Train ADJ-R2 : 0.9528487605052167
f statistics : 1704.4625441838434 | f_pvalue : 0.0
----------------------------------------------------------------------------------------------------
for site CIGS1-001
R2 For Vmp (V) : 
Test R2 : 0.935207471779459 | Train R2 : 0.9362047191371987
Test ADJ-R2 : 0.9347403302638098 | Train ADJ-R2 : 0.9360904316401317
f statistics : 1247.838497598502 | f_pvalue : 0.0
----------------------------------------------------------------------------------------------------
for site aSiTandem90-31
R2 For Vmp (V) : 
Test R2 : 0.4589788147924604 | Train R2 : 0.5151282920633443
Test ADJ-R2 : 0.45510884923303596 | Train ADJ-R2 : 0.5142664511320669
f statistics : 91.05595651895628 | f_pvalue : 0.0
----------------------------------------------------------------------------------------------------
for site xSi11246
R2 For Vmp (V) : 
Test R

In [16]:
reg_model=Pipeline([
    ("preproc", preprocessor),
    ("poly",PolynomialFeatures(degree=2)),
    ("model", LinearRegression())
])
v_reg_model=Pipeline([
    ("preproc", v_preprocessor),
    # ("poly",PolynomialFeatures(degree=2)),
    ("model", XGBRegressor(**l1_xg_boost_params)) #XGBRegressor(**l2_xg_boost_params)
])
multi_train(reg_model,v_reg_model)

for site mSi0247
R2 For Vmp (V) : 
Test R2 : 0.9710387146832818 | Train R2 : 0.9755801652607007
Test ADJ-R2 : 0.9708277803401156 | Train ADJ-R2 : 0.975535982458567
----------------------------------------------------------------------------------------------------
for site CIGS1-001
R2 For Vmp (V) : 
Test R2 : 0.9718295513968819 | Train R2 : 0.9730205113177959
Test ADJ-R2 : 0.9716264479462465 | Train ADJ-R2 : 0.9729721783033168
----------------------------------------------------------------------------------------------------
for site aSiTandem90-31
R2 For Vmp (V) : 
Test R2 : 0.5221660548884302 | Train R2 : 0.6533217012123513
Test ADJ-R2 : 0.5187480724484332 | Train ADJ-R2 : 0.652705493784716
----------------------------------------------------------------------------------------------------
for site xSi11246
R2 For Vmp (V) : 
Test R2 : 0.7630635910531272 | Train R2 : 0.8764900139731507
Test ADJ-R2 : 0.7613391630404134 | Train ADJ-R2 : 0.8762666282143755
-----------------------------

In [17]:
v_features=["GHI","DNI","POA_CMP22","Cab_Temp","Dry_Temp","Mod_Temp","Pressure","RH","QA_residual"]

preprocessor = ColumnTransformer([
    ("norm", StandardScaler(),v_features)
])
def single_train(reg_model,stat_rep=False):
    vmp_test_r2_ls=[]
    vmp_train_r2_ls=[]
    adj_vmp_test_r2_ls=[]
    adj_vmp_train_r2_ls=[]
    for solar_model in x_train_sites:
        x_tr=x_train_sites[solar_model]
        x_te=x_test_sites[solar_model]
        y_tr=y_train_sites[solar_model]
        y_te=y_test_sites[solar_model]

        reg_model.fit(x_tr[v_features], y_tr["Vmp (V)"])
    
        print(f"for site {solar_model}")
        
        vmp_test_r2,vmp_train_r2,adj_vmp_train_r2,adj_vmp_test_r2=evaluate(reg_model,x_tr[v_features],y_tr[["Vmp (V)"]],x_te[v_features],y_te[["Vmp (V)"]])
        vmp_test_r2_ls.append(vmp_test_r2)
        vmp_train_r2_ls.append(vmp_train_r2)
        adj_vmp_train_r2_ls.append(adj_vmp_train_r2)
        adj_vmp_test_r2_ls.append(adj_vmp_test_r2)
        if(stat_rep):
            reg_model.named_steps["model"].get_f_test()

        if(solar_model == "aSiTandem90-31" and stat_rep):
            names=get_f_names(reg_model)
            print(reg_model.named_steps["model"].get_summary(names))
            
        print("-"*100)

    print(f"mean test r2 : {np.mean(vmp_test_r2_ls)} | mean train r2 :{np.mean(vmp_train_r2_ls)}")
    print(f"mean test adj-r2 : {np.mean(adj_vmp_test_r2_ls)} | mean train adj-r2 :{np.mean(adj_vmp_train_r2_ls)}") 

In [31]:
reg_model=Pipeline([
    ("preproc", preprocessor),
    ("poly",PolynomialFeatures(degree=2)),
    ("model", StatsModelsOLS()) #XGBRegressor(**l2_xg_boost_params)
])
single_train(reg_model,stat_rep=True)

for site mSi0247
R2 For Vmp (V) : 
Test R2 : 0.9335155979665721 | Train R2 : 0.9230628440229718
Test ADJ-R2 : 0.9330801106170081 | Train ADJ-R2 : 0.9229375845432697
f statistics : 1218.2028954781945 | f_pvalue : 0.0
----------------------------------------------------------------------------------------------------
for site CIGS1-001
R2 For Vmp (V) : 
Test R2 : 0.8957030864912889 | Train R2 : 0.8971999363898134
Test ADJ-R2 : 0.8950268096745898 | Train ADJ-R2 : 0.8970342189310114
f statistics : 895.06596289449 | f_pvalue : 0.0
----------------------------------------------------------------------------------------------------
for site aSiTandem90-31
R2 For Vmp (V) : 
Test R2 : 0.45007194083651403 | Train R2 : 0.4979513237129337
Test ADJ-R2 : 0.4465341620427532 | Train ADJ-R2 : 0.49714833133927394
f statistics : 102.52673732614367 | f_pvalue : 0.0
                            OLS Regression Results                            
Dep. Variable:                Vmp (V)   R-squared:             

In [18]:
reg_model=Pipeline([
    ("preproc", preprocessor),
    # ("poly",PolynomialFeatures(degree=2)),
    ("model", XGBRegressor(**l1_xg_boost_params)) #XGBRegressor(**l2_xg_boost_params)
])
single_train(reg_model)

for site mSi0247
R2 For Vmp (V) : 
Test R2 : 0.9687501756771627 | Train R2 : 0.9740126215795535
Test ADJ-R2 : 0.9685454825047424 | Train ADJ-R2 : 0.9739703121718501
----------------------------------------------------------------------------------------------------
for site CIGS1-001
R2 For Vmp (V) : 
Test R2 : 0.9668904215781657 | Train R2 : 0.9691280419455031
Test ADJ-R2 : 0.9666757341100126 | Train ADJ-R2 : 0.9690782752210735
----------------------------------------------------------------------------------------------------
for site aSiTandem90-31
R2 For Vmp (V) : 
Test R2 : 0.5248470264483382 | Train R2 : 0.6365279173519598
Test ADJ-R2 : 0.5217902882339245 | Train ADJ-R2 : 0.6359465687214583
----------------------------------------------------------------------------------------------------
for site xSi11246
R2 For Vmp (V) : 
Test R2 : 0.7606416204937476 | Train R2 : 0.8777849561481967
Test ADJ-R2 : 0.7590749111006158 | Train ADJ-R2 : 0.8775860528218556
---------------------------